In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [3]:
file = '../data/jla_binned_distances_31nodes_v1.txt'

# filefull = '/home/cosmocicatais/Documents/github/neuralCosmoReconstruction/data/jla_lcparams.txt'
syscov = np.loadtxt('../data/cov_jla_binned_distances_31nodes_v1.txt',skiprows=1).reshape((31,31))
# syscov = np.loadtxt('../data/jla_v0_covmatrix.dat',skiprows=1).reshape((740,740))
# data = pd.read_csv(file2, names=['z', '$D_L$'], sep = " ", skiprows=3)
# data = np.loadtxt(file, usecols=[1,4], skiprows=1)
data = np.loadtxt(file, skiprows=3)
nsize = len(syscov)
# print(data.head(31))
# data.head(31) 
# plt.savefig("DistbinnedJLA.png")
np.shape(syscov)

errors = np.diag(syscov).reshape(nsize,1)
errors = np.where(errors!= 0, np.log(errors), 0)
np.shape(errors)
len(data), len(errors)

(31, 31)

In [4]:
shuffle = [0,1,2,3,4,5,6,7,8,30,28,26,24,22,20,9,10,12,14,16,18,29,27,25,23,21,19,17,15,13,11]
data = data[shuffle]

errors= errors[shuffle]
len(data), len(errors), len(shuffle)

(31, 31, 31)

In [5]:
dl = data[:,1]
z = data[:,0]
np.shape(z), np.shape(dl), np.shape(errors), len(data)

((31,), (31,), (31, 1), 31)

In [6]:
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]

zz = np.log(z)
# scalerz = StandardScaler()
# scalerz.fit(z.reshape(-1, 1))
# zz = scalerz.transform(z.reshape((-1,1)))

# scalerdl = StandardScaler()
# scalerdl.fit(dl.reshape(-1, 1))
# dll = scalerdl.transform(dl.reshape((-1,1)))
dll = dl.reshape(nsize, 1)
# scalererr = StandardScaler()
# scalererr.fit(errors.reshape(-1, 1))
# errscl = scalererr.transform(errors.reshape((-1,1)))

z_train, z_test = np.split(zz, indx)
y = np.concatenate((dll, errors), axis=1)
y_train, y_test = np.split(y, indx)

In [7]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([50, 100]))
# HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
# HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([1, 4, 8]))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([1, 4]))

In [8]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [9]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [10]:
def hzmodel(input_z, hparams):
    #encoder
    efirst = Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(1,))(input_z)
    x = Dense(hparams[HP_NUM_UNITS2], activation='relu')(efirst)
    x = Dense(hparams[HP_NUM_UNITS3], activation='relu')(x)
#     x = Dense(hparams[HP_NUM_UNITS4], activation='relu')(x)
#     ehidden2 = Dense(hparams[HP_NUM_UNITS3], activation='relu')(ehidden1)
    elast = Dense(2, activation='linear')(x)
        
    return elast

In [11]:
epochs = 300
input_z = tf.keras.layers.Input(shape = (1,))

In [12]:
def train_test_model(hparams):
    model = Model(input_z, hzmodel(input_z, hparams))  
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(z_train, y_train, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], verbose=0) 
    _, loss = model.evaluate(z_test, y_test)


    return loss
    

In [13]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [14]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
#                 for num_units4 in HP_NUM_UNITS4.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#                     for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                  HP_NUM_UNITS1: num_units1,
                  HP_NUM_UNITS2: num_units2,
                  HP_NUM_UNITS3: num_units3,
#                       HP_NUM_UNITS4: num_units4,
                #                       HP_DROPOUT: dropout_rate,
    #                       HP_OPTIMIZER: optimizer,
                  HP_BATCHSIZE: batch_size
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
    #                 print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams)
                session_num += 1

print("Total combinations: {}".format(session_num+1))

--- Starting trial: run-0
1/1 [==============================] - 0s 69ms/step - loss: 0.0556 - mean_squared_error: 0.0556
--- Starting trial: run-1
1/1 [==============================] - 0s 62ms/step - loss: 0.0856 - mean_squared_error: 0.0856
--- Starting trial: run-2
1/1 [==============================] - 0s 53ms/step - loss: 0.1604 - mean_squared_error: 0.1604
--- Starting trial: run-3
1/1 [==============================] - 0s 55ms/step - loss: 0.0705 - mean_squared_error: 0.0705
--- Starting trial: run-4
1/1 [==============================] - 0s 54ms/step - loss: 0.1166 - mean_squared_error: 0.1166
--- Starting trial: run-5
1/1 [==============================] - 0s 55ms/step - loss: 0.1251 - mean_squared_error: 0.1251
--- Starting trial: run-6
1/1 [==============================] - 0s 56ms/step - loss: 0.1154 - mean_squared_error: 0.1154
--- Starting trial: run-7
1/1 [==============================] - 0s 54ms/step - loss: 0.0782 - mean_squared_error: 0.0782
--- Starting trial: run-

1/1 [==============================] - 0s 53ms/step - loss: 0.1269 - mean_squared_error: 0.1269
--- Starting trial: run-14
1/1 [==============================] - 0s 52ms/step - loss: 0.0900 - mean_squared_error: 0.0900
--- Starting trial: run-15
1/1 [==============================] - 0s 54ms/step - loss: 0.0919 - mean_squared_error: 0.0919
--- Starting trial: run-16
1/1 [==============================] - 0s 54ms/step - loss: 0.1022 - mean_squared_error: 0.1022
--- Starting trial: run-17
1/1 [==============================] - 0s 57ms/step - loss: 1.3194 - mean_squared_error: 1.3194
--- Starting trial: run-18
1/1 [==============================] - 0s 56ms/step - loss: 0.0894 - mean_squared_error: 0.0894
--- Starting trial: run-19
1/1 [==============================] - 0s 53ms/step - loss: 0.0334 - mean_squared_error: 0.0334
--- Starting trial: run-20
1/1 [==============================] - 0s 52ms/step - loss: 0.2579 - mean_squared_error: 0.2579
--- Starting trial: run-21
1/1 [===========

1/1 [==============================] - 0s 53ms/step - loss: 0.2133 - mean_squared_error: 0.2133
--- Starting trial: run-23
1/1 [==============================] - 0s 54ms/step - loss: 0.1721 - mean_squared_error: 0.1721
--- Starting trial: run-24
1/1 [==============================] - 0s 52ms/step - loss: 0.0701 - mean_squared_error: 0.0701
--- Starting trial: run-25
1/1 [==============================] - 0s 54ms/step - loss: 0.3960 - mean_squared_error: 0.3960
--- Starting trial: run-26
1/1 [==============================] - 0s 53ms/step - loss: 0.0971 - mean_squared_error: 0.0971
--- Starting trial: run-27
1/1 [==============================] - 0s 53ms/step - loss: 0.1657 - mean_squared_error: 0.1657
--- Starting trial: run-28
1/1 [==============================] - 0s 54ms/step - loss: 0.1134 - mean_squared_error: 0.1134
--- Starting trial: run-29
1/1 [==============================] - 0s 54ms/step - loss: 0.4518 - mean_squared_error: 0.4518
--- Starting trial: run-30
1/1 [===========

1/1 [==============================] - 0s 53ms/step - loss: 0.0985 - mean_squared_error: 0.0985
--- Starting trial: run-33
1/1 [==============================] - 0s 67ms/step - loss: 0.0800 - mean_squared_error: 0.0800
--- Starting trial: run-34
1/1 [==============================] - 0s 62ms/step - loss: 0.0470 - mean_squared_error: 0.0470
--- Starting trial: run-35
1/1 [==============================] - 0s 67ms/step - loss: 0.0472 - mean_squared_error: 0.0472
--- Starting trial: run-36
1/1 [==============================] - 0s 62ms/step - loss: 0.0544 - mean_squared_error: 0.0544
--- Starting trial: run-37
1/1 [==============================] - 0s 59ms/step - loss: 0.0788 - mean_squared_error: 0.0788
--- Starting trial: run-38
1/1 [==============================] - 0s 53ms/step - loss: 0.0626 - mean_squared_error: 0.0626
--- Starting trial: run-39
1/1 [==============================] - 0s 52ms/step - loss: 0.0762 - mean_squared_error: 0.0762
--- Starting trial: run-40
1/1 [===========

1/1 [==============================] - 0s 53ms/step - loss: 0.1474 - mean_squared_error: 0.1474
--- Starting trial: run-43
1/1 [==============================] - 0s 55ms/step - loss: 0.1122 - mean_squared_error: 0.1122
--- Starting trial: run-44
1/1 [==============================] - 0s 54ms/step - loss: 0.1416 - mean_squared_error: 0.1416
--- Starting trial: run-45
1/1 [==============================] - 0s 53ms/step - loss: 0.0622 - mean_squared_error: 0.0622
--- Starting trial: run-46
1/1 [==============================] - 0s 61ms/step - loss: 0.0757 - mean_squared_error: 0.0757
--- Starting trial: run-47
1/1 [==============================] - 0s 56ms/step - loss: 0.1447 - mean_squared_error: 0.1447
--- Starting trial: run-48
1/1 [==============================] - 0s 53ms/step - loss: 0.3883 - mean_squared_error: 0.3883
--- Starting trial: run-49
1/1 [==============================] - 0s 55ms/step - loss: 0.0533 - mean_squared_error: 0.0533
--- Starting trial: run-50
1/1 [===========

...

1/1 [==============================] - 0s 53ms/step - loss: 0.0474 - mean_squared_error: 0.0474
--- Starting trial: run-83
1/1 [==============================] - 0s 56ms/step - loss: 0.0336 - mean_squared_error: 0.0336
--- Starting trial: run-84
1/1 [==============================] - 0s 173ms/step - loss: 0.1012 - mean_squared_error: 0.1012
--- Starting trial: run-85
1/1 [==============================] - 0s 54ms/step - loss: 0.0575 - mean_squared_error: 0.0575
--- Starting trial: run-86
1/1 [==============================] - 0s 69ms/step - loss: 0.0549 - mean_squared_error: 0.0549
--- Starting trial: run-87
1/1 [==============================] - 0s 53ms/step - loss: 0.0509 - mean_squared_error: 0.0509
--- Starting trial: run-88
1/1 [==============================] - 0s 54ms/step - loss: 0.0628 - mean_squared_error: 0.0628
--- Starting trial: run-89
1/1 [==============================] - 0s 55ms/step - loss: 0.0487 - mean_squared_error: 0.0487
--- Starting trial: run-90
1/1 [==========

1/1 [==============================] - 0s 53ms/step - loss: 0.0622 - mean_squared_error: 0.0622
--- Starting trial: run-93
1/1 [==============================] - 0s 54ms/step - loss: 0.0551 - mean_squared_error: 0.0551
--- Starting trial: run-94
1/1 [==============================] - 0s 52ms/step - loss: 0.0513 - mean_squared_error: 0.0513
--- Starting trial: run-95
1/1 [==============================] - 0s 69ms/step - loss: 0.0695 - mean_squared_error: 0.0695
--- Starting trial: run-96
1/1 [==============================] - 0s 53ms/step - loss: 0.2333 - mean_squared_error: 0.2333
--- Starting trial: run-97
1/1 [==============================] - 0s 53ms/step - loss: 0.3983 - mean_squared_error: 0.3983
--- Starting trial: run-98
1/1 [==============================] - 0s 53ms/step - loss: 0.0468 - mean_squared_error: 0.0468
--- Starting trial: run-99
1/1 [==============================] - 0s 52ms/step - loss: 0.0510 - mean_squared_error: 0.0510
--- Starting trial: run-100
1/1 [==========

1/1 [==============================] - 0s 53ms/step - loss: 0.0574 - mean_squared_error: 0.0574
--- Starting trial: run-103
1/1 [==============================] - 0s 54ms/step - loss: 0.0440 - mean_squared_error: 0.0440
--- Starting trial: run-104
1/1 [==============================] - 0s 53ms/step - loss: 0.0547 - mean_squared_error: 0.0547
--- Starting trial: run-105
1/1 [==============================] - 0s 52ms/step - loss: 0.0384 - mean_squared_error: 0.0384
--- Starting trial: run-106
1/1 [==============================] - 0s 53ms/step - loss: 0.0528 - mean_squared_error: 0.0528
--- Starting trial: run-107
1/1 [==============================] - 0s 53ms/step - loss: 0.1364 - mean_squared_error: 0.1364
--- Starting trial: run-108
1/1 [==============================] - 0s 54ms/step - loss: 0.0520 - mean_squared_error: 0.0520
--- Starting trial: run-109
1/1 [==============================] - 0s 54ms/step - loss: 0.0431 - mean_squared_error: 0.0431
--- Starting trial: run-110
1/1 [===

1/1 [==============================] - 0s 52ms/step - loss: 0.0494 - mean_squared_error: 0.0494
--- Starting trial: run-113
1/1 [==============================] - 0s 53ms/step - loss: 0.0495 - mean_squared_error: 0.0495
--- Starting trial: run-114
1/1 [==============================] - 0s 53ms/step - loss: 0.0460 - mean_squared_error: 0.0460
--- Starting trial: run-115
1/1 [==============================] - 0s 53ms/step - loss: 0.0401 - mean_squared_error: 0.0401
--- Starting trial: run-116
1/1 [==============================] - 0s 55ms/step - loss: 0.0425 - mean_squared_error: 0.0425
--- Starting trial: run-117
1/1 [==============================] - 0s 56ms/step - loss: 0.0673 - mean_squared_error: 0.0673
--- Starting trial: run-118
1/1 [==============================] - 0s 54ms/step - loss: 0.0568 - mean_squared_error: 0.0568
--- Starting trial: run-119
1/1 [==============================] - 0s 54ms/step - loss: 0.0379 - mean_squared_error: 0.0379
--- Starting trial: run-120
1/1 [===

1/1 [==============================] - 0s 52ms/step - loss: 0.0588 - mean_squared_error: 0.0588
--- Starting trial: run-123
1/1 [==============================] - 0s 56ms/step - loss: 0.0653 - mean_squared_error: 0.0653
--- Starting trial: run-124
1/1 [==============================] - 0s 57ms/step - loss: 0.0459 - mean_squared_error: 0.0459
--- Starting trial: run-125
1/1 [==============================] - 0s 55ms/step - loss: 0.0554 - mean_squared_error: 0.0554
--- Starting trial: run-126
1/1 [==============================] - 0s 56ms/step - loss: 0.1479 - mean_squared_error: 0.1479
--- Starting trial: run-127
1/1 [==============================] - 0s 55ms/step - loss: 0.0551 - mean_squared_error: 0.0551
Total combinations: 129


In [15]:
# !kill 7439
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 5351), started 1:58:56 ago. (Use '!kill 5351' to kill it.)